# JOSAA-scrape
A python script to scrape the JOSAA OR/CR page at the [JOSAA Result Archieve](https://josaa.nic.in/Result/Result/OpeningClosingRankArchieve.aspx).

Uses `requests`, `BeautifulSoup` and `pandas`

I do not own this data, nor am I liable for any consequences of its usage.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

`params` is a dictionary containing params info from DevTools > Network > Params

It is an ASP .NET powered form, so a `__VIEWSTATE` and `__EVENTVALIDATION` are assigned to every session. Every dropdown selection adds some encoded info to these parameters, so it is not possible to send them all at once.

We can get these values from the hidden `__VIEWSTATE` and similar inputs on the page after every POST request.

The `josaa_scrape()` function return a pandas dataframe containing the year and round specific OR/CR.

`pandas` converts the OR/CR to `float`s, so I typecast them back as `int`s.

2016 only had 6 rounds for some reason, so the ugly adjustments were required in `years` and `rounds`.

In [2]:
old_year_url = 'https://josaa.admissions.nic.in/applicant/seatmatrix/openingclosingrankarchieve.aspx'
curr_year_url = 'https://josaa.admissions.nic.in/applicant/SeatAllotmentResult/CurrentORCR.aspx'

params = {
    "ctl00$ContentPlaceHolder1$ddlInstype": "ALL",
    "ctl00$ContentPlaceHolder1$ddlInstitute": "ALL",
    "ctl00$ContentPlaceHolder1$ddlBranch": "ALL",
    "ctl00$ContentPlaceHolder1$ddlSeattype": "ALL",
    "ctl00$ContentPlaceHolder1$btnSubmit": "Submit"
}

In [3]:
years = [
    "2024"
    # "2023",
    # "2022",
    # "2021"
    # "2018",
    # "2017",
    # "2016"
]

rounds = [
    "1",
    "2",
    "3",
    "4",
    "5",
    # "6",
    # "7"
]

In [4]:
def josaa_scrape(year, round, isCurrYear):
    """
    Sample usage: df = josaa_scrape("2024", "1", True)
    Sample usage: df = josaa_scrape("2023", "1", False)
    df.info()
    """
    with requests.Session() as s:
        url = curr_year_url if isCurrYear else old_year_url
        R = s.get(url)
        data = {}

        if not isCurrYear:
            data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
            data["ctl00$ContentPlaceHolder1$ddlYear"] = year
            R = s.post(url, data=data)

        data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
        data["ctl00$ContentPlaceHolder1$ddlroundno"] = round
        R = s.post(url, data=data)

        for key, value in params.items():
            data.update({tag['name']: tag['value'] for tag in BeautifulSoup(R.content, 'lxml').select('input[name^=__]')})
            data[key] = value
            R = s.post(url, data=data)

    table = BeautifulSoup(R.text, 'lxml').find(id='ctl00_ContentPlaceHolder1_GridView1')
    df = pd.read_html(table.prettify())[0]
    df.dropna(inplace=True, how="all")

    df["Year"] = year
    df["Round"] = round
    # code commented because PwD rank contains P at end, e.g. 150P
    # df['Opening Rank'] = df['Opening Rank'].astype(int) 
    # df['Closing Rank'] = df['Closing Rank'].astype(int)
    df['Opening Rank'] = df['Opening Rank']
    df['Closing Rank'] = df['Closing Rank']

    return df

In [5]:
for year in years:
    for Round in rounds:
        print(Round, year)
        data = josaa_scrape(year, Round, True)
        print(data.shape)
        data.to_csv(path_or_buf= year + "-" + Round + ".csv", index=False)

3 2024


/tmp/ipykernel_84553/1908411863.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.prettify())[0]


(11307, 9)
4 2024


/tmp/ipykernel_84553/1908411863.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.prettify())[0]


(11281, 9)
5 2024


/tmp/ipykernel_84553/1908411863.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.prettify())[0]


(11261, 9)
